### StructuredOutputParser
LLM의 답변을 dict 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할때 유용.

#### **장점**
Pydantic/JSON 파서가 더 강력하다는 평가. StructuredOutputParser는 로컬 모델과 같은 덜 강력한 모델에서도 유용.
GPT나 Claude 모델 보다 인텔리전스가 낮은(즉, parameter 수가 적은)모델에서 특히 효과적.

#### **참고 사항**
로컬 모델의 경우 Pydantic 파서가 동작하지 않는 상황 빈번하게 발생. 대안으로 StructuredOutputParser를 사용하는 것이 좋은 해결책.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from langchain_teddynote import logging
logging.langsmith("Sturctured")

LangSmith 추적을 시작합니다.
[프로젝트명]
Sturctured


In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [16]:
response_schmea = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name = "source",
        description="사용자의 질문에 답하기 위해 사용된 '출처','웹사이트주소' 이여야 합니다.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schmea)

In [17]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 '출처','웹사이트주소' 이여야 합니다.
}
```


In [18]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template = "answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [22]:
llm = ChatOpenAI(
    temperature=0,
)
chain = prompt | llm | output_parser

In [23]:
answer = chain.invoke({"question": "대한민국의 수도"})

In [24]:
print(answer)

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}


In [26]:
answer['answer']

'서울'

In [25]:
for s in chain.stream({"question":"세종대왕의 업적은 무엇인가요?"}):
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화 발전에 큰 기여를 한 대한민국의 역사적 인물입니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
